使用内置的 Inception V3 模型。

In [5]:
from keras.applications import inception_v3
from keras import backend as K

# 禁用所有与训练有关的操作
K.set_learning_phase(0)

# 不包含全连接层的 Inception V3 网络，并使用预训练的 ImageNet 权重来加载模型
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

更靠近底层的生成的是几何图形，更靠近上层的是 ImageNet 类别。

将每个层次的损失贡献度和层次的名称映射起来：

In [6]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

定义需要最大化的损失：（把上面的各层损失贡献度加权求和）

In [7]:
import tensorflow as tf

layer_dict = dict([(layer.name, layer) for layer in model.layers])

with tf.GradientTape() as gTape:
    loss = K.variable(0.)

    for layer_name in layer_contributions:
        coeff = layer_contributions[layer_name]
        activation = layer_dict[layer_name].output
    
        scaling = K.prod(K.cast(K.shape(activation), 'float32'))
        loss.assign_add(coeff * K.sum(K.square(activation[:, 2: -2, -2, :])) / scaling)
    

In [8]:

dream = model.input

grads = K.gradients(loss, dream)
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]

fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...loss value at ', i, ':', loss_value)
        x += step * grad_values
    return x


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.